In [55]:
import pandas as pd

In [56]:
def payer_category(x):
    payers = {
        'Gross Charge': 'gross',
        'Maximum Contracted Payment': 'max',
        'Minimum Contracted Payment': 'min',
        'Self Pay': 'cash',
    }
    
    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [57]:
file = '411889892_samedaysurgerycenterllc_standardcharges.xlsx'
df = pd.read_excel(file, dtype={'Px Code': str, 'CPT(R)/HCPCS Code': str})

In [58]:
df.rename(columns={
    'Px Code': 'internal_code',
    'Procedure Description': 'description',
    'CPT(R)/HCPCS Code': 'code'
}, inplace=True)

In [59]:
# Skip first row
df = df[1:]

In [60]:
cols = df.columns.tolist()
df = pd.melt(df, id_vars=cols[:3], value_vars=cols[3:], var_name='payer', value_name='standard_charge')

In [61]:
# df['payer_name'] = df['payer_orig']
df['payer_category'] = df['payer_orig'].apply(payer_category)
df['standard_charge_desc'] = 'Subject to multiple procedure discounting'
# df['code_orig'] = df['code']
df.loc[df['code'].notnull(), 'hcpcs_cpt'] = df['code']

In [65]:
# Cleaning
df['standard_charge'] = df['standard_charge'].str.replace(',', '')

# Null filling
df['code'].fillna('na', inplace=True)
# df['code_prefix'].fillna('none', inplace=True)



df.dropna(subset=['standard_charge'], inplace=True)

In [63]:
# df['hospital_ein'] = '41-1889892'
# df['hospital_ccn'] = '430093'
# df['filename'] = file
# df['file_last_updated'] = '2022-12-01'
df['url'] = 'http://www.samedaysurgerycenter.org/uploads/411889892_samedaysurgerycenterllc_standardcharges.xlsx'

In [64]:
df.to_csv('sameday_surgery.csv', index=False)